# CDMS Helpful Features<a id='top' class="tocSkip"> </a>



© The CDAT software was developed by LLNL. This tutorial was written by Denis Nadeau. This work was performed under the auspices of the U.S. Department of Energy by Lawrence Livermore National Laboratory under Contract DE-AC52-07NA27344.

[Download the Jupyter Notebook](CDMS2 Helpful Features.ipynb)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Create-a-regular-Grid" data-toc-modified-id="Create-a-regular-Grid-1">Create a regular Grid</a></span></li><li><span><a href="#Get--1D-weights" data-toc-modified-id="Get--1D-weights-2">Get  1D weights</a></span></li><li><span><a href="#Compute-the-2D-weights" data-toc-modified-id="Compute-the-2D-weights-3">Compute the 2D weights</a></span></li><li><span><a href="#Create-a-cdms2-variable-with-some-extra-metadata" data-toc-modified-id="Create-a-cdms2-variable-with-some-extra-metadata-4">Create a cdms2 variable with some extra metadata</a></span></li><li><span><a href="#Save-a-variable-to-an-nc-file" data-toc-modified-id="Save-a-variable-to-an-nc-file-5">Save a variable to an nc file</a></span></li></ul></div>

# Create a regular Grid

[Back to Top](#top)

In [1]:
#!/usr/bin/env python

# Generate the weights of a regular/uniform grid, and store the
# variable in a netcdf file

# J-Y Peterschmitt - LSCE - 05/2019

import numpy as np, time, os
import cdms2

In [2]:
# Define and generate the grid. The cell bounds are automatically
# generated and will be automatically used for computing the cell weights

# inc = 0.5
# nb_lat = 360
inc = 1
nb_lat = 180
startLat, nlat, deltaLat, startLon, nlon, deltaLon = (-90 + inc/2., nb_lat, inc,
                                                      -180 + inc/2., 2*nb_lat, inc)

uniform_grid = cdms2.createUniformGrid(startLat, nlat, deltaLat, startLon, nlon, deltaLon)


/export/reshel3/anaconda52/envs/cdms2/lib/python2.7/site-packages/cdms2/axis.py:1683: UserWarning: 
Your first bounds[0,0] -180.000000000000000 will be corrected to -180.000000000000000
Your bounds bounds[-1,1] 180.000000000000000 will be corrected to 180.000000000000000
  warnings.warn(msg, UserWarning)


In [3]:
# Check the generated grid
print('Grid information:')
print(uniform_grid)

Grid information:
Grid has Python id 0x7f0c2625dd90.
Gridtype: uniform
Grid shape: (180, 360)
Order: yx


In [4]:
# Get the grid axes
lat_axis = uniform_grid.getLatitude()
lon_axis = uniform_grid.getLongitude()

print('\nLongitude information:')
print(lon_axis)

print('\nLongitude bounds information:')
print(lon_axis.getBounds()[:3, :])
print(lon_axis.getBounds()[-3:, :])

print('\nLatitude information:')
print(lat_axis)

print('\nLatitude bounds information:')
print(lat_axis.getBounds()[:3, :])
print(lat_axis.getBounds()[-3:, :])


Longitude information:
   id: longitude
   Designated a longitude axis.
   units:  degrees_east
   Length: 360
   First:  -179.5
   Last:   179.5
   Other axis attributes:
      axis: X
      modulo: 360.0
      topology: circular
   Python id:  0x7f0c1d119450


Longitude bounds information:
[[-180. -179.]
 [-179. -178.]
 [-178. -177.]]
[[177. 178.]
 [178. 179.]
 [179. 180.]]

Latitude information:
   id: latitude
   Designated a latitude axis.
   units:  degrees_north
   Length: 180
   First:  -89.5
   Last:   89.5
   Other axis attributes:
      axis: Y
   Python id:  0x7f0c1d119710


Latitude bounds information:
[[-90. -89.]
 [-89. -88.]
 [-88. -87.]]
[[87. 88.]
 [88. 89.]
 [89. 90.]]


# Get  1D weights
[Back to Top](#top)


In [5]:
# Get the axes' weights and combine them into a 2D variable
lat_weights, lon_weights = uniform_grid.getWeights()
grid_weights = np.outer(lat_weights, lon_weights)

print('\nCheck that the sum of the generated weights is 1.0:')
print('  lat_weights.sum() =', lat_weights.sum())
print('  lon_weights.sum() =', lon_weights.sum())
print('  grid_weights.sum() =', grid_weights.sum())


Check that the sum of the generated weights is 1.0:
('  lat_weights.sum() =', 1.0)
('  lon_weights.sum() =', 1.0)
('  grid_weights.sum() =', 1.0)


# Compute the 2D weights
[Back to Top](#top)


In [10]:
# Combine the 2D weights var with the axes to create a variable
weight_var = cdms2.createVariable(grid_weights, axes=[lat_axis,
                                                      lon_axis] , id='cell_weight')

# Create a cdms2 variable with some extra metadata
[Back to Top](#top)


In [11]:
# Easily add some extra metadata
weight_var.units = '1'
weight_var.history = 'File generated with cdms2 on ' + time.asctime()

# Save a variable to an nc file
[Back to Top](#top)


In [12]:
nc_out = './weight_file.nc'
f_out = cdms2.open(nc_out, 'w')
f_out.write(weight_var)
f_out.close()

print('\nSuccessfully saved the weights to ' + nc_out)

print('\nChecking the header of the generated file\n')
os.system('ncdump -h '+ nc_out)



Successfully saved the weights to ./weight_file.nc

Checking the header of the generated file



0